# Notebook to automatize the writing of topological ferroelectric structures

## Author: Marti Checa

### Publication "On demand nanoengineering of local topological ferroelectric structures"

### Imports

In [ ]:
from sys import exit
import time
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from nifpga import Session
import time
import pylab as pl
from IPython import display
import win32com.client  
import os
from joblib import Parallel, delayed
import pythoncom
import sidpy as sid
from skimage.registration import phase_cross_correlation
from scipy.ndimage import fourier_shift
import warnings
warnings.filterwarnings('ignore')

import h5py
import pyNSID

from scipy.special import erf

# import acquition.py
from Acquisition_v0_2 import Acquisition   # include the Acquistion_v0.py in the same directory

### Start executable - activeX (LABView) --> needs it running before it can connect

In [ ]:
exe_path = r'C:\Users\Asylum User\Desktop\FPGA Programs\Marti Checa\Spirals for Pascal\BEPyAE 051123 01\\BEPyAE.exe'
os.startfile(exe_path)
time.sleep(1)

### Connect to executable via active X

In [ ]:
labview = win32com.client.Dispatch("BEPyAE.Application")
#slash is critical here: double slash, tilt to left
VI = labview.getvireference(r'C:\Users\Asylum User\Desktop\FPGA Programs\Marti Checa\Spirals for Pascal\BEPyAE 051123 01\\BEPyAE.exe\\FPGA PyScanner\\FPGA_PyScanner_01.vi') 

## Define Important functions

In [ ]:
def move_tip_to_pos(fin_x,fin_y,trans_t):
    line_scan_cluster = list(VI.getcontrolvalue('line_scan_control_cluster'))#get the line scan cluster
    line_scan_cluster[2] = fin_x 
    line_scan_cluster[3] = fin_y 
    line_scan_cluster[4] = trans_t 
    VI.setcontrolvalue('line_scan_control_cluster',tuple(line_scan_cluster))#send the new parameters
    VI.setcontrolvalue('make_cur_pos_start_pos', True);#tell it to use current pos as starting pos
    VI.setcontrolvalue('do_probe_move_update', True);#update params of move tip
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_probe_move_update')        
    VI.setcontrolvalue('do_probe_move', True) #Move the tip to new location
    moving = True
    while (moving==True):
        moving = VI.getcontrolvalue('do_probe_move')
    
    print('Tip moved to ('+str(fin_x) + ',' + str(fin_y) + ')' )
       
def change_tip_bias(tip_bias):        
    VI.setcontrolvalue('set_AO2_V',tip_bias); # changed value
    #VI.setcontrolvalue('set_AO0_V',VI.getcontrolvalue('get_AO0_V')); # changed value
    #VI.setcontrolvalue('set_AO1_V',VI.getcontrolvalue('get_AO1_V')); # changed value
    VI.setcontrolvalue('do_set_output_val', 1);
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_set_output_val')
        
    print('Tip bias changed to '+str(tip_bias)+'V')
        
def do_write_spiral_scan(center_x,center_y,hh_writting_bias,
                   hh_sp_number,hh_Vx,hh_Vy,hh_cycles,
                   hh_spiral_time,hh_direction,hh_return):
    
    print("Performing spiral scans...")
    #Defininig spiral scan type
    spiral_scan_cluster = list(VI.getcontrolvalue('spiral_scan_control_cluster'))#get the spiral raster scan cluster
    spiral_scan_cluster[1] = hh_Vx #change X size
    spiral_scan_cluster[3] = hh_Vy #change X size
    spiral_scan_cluster[4] = hh_cycles #change cycles
    spiral_scan_cluster[5] = hh_spiral_time #change duration
    spiral_scan_cluster[7] = hh_direction #change direction
    spiral_scan_cluster[8] = hh_return #change return type
    VI.setcontrolvalue('spiral_scan_control_cluster',tuple(spiral_scan_cluster))#send the new parameters
    
    #set up spiral center
    VI.setcontrolvalue('scan_x_offset_V', center_x)
    VI.setcontrolvalue('scan_y_offset_V', center_y)
    VI.setcontrolvalue('set_AO0_V',center_x); # changed value
    VI.setcontrolvalue('set_AO1_V',center_y); # changed value
    VI.setcontrolvalue('scan_type', 1)#change to spiral scan
    
    #update params
    VI.setcontrolvalue('do_scan_update', 1) #need to tell PyScanner to update parameters internally 
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_scan_update')
        
    TipBias_vec=np.full((int(hh_spiral_time*1E6)),hh_writting_bias)#generating vector of tip biases
    VI.setcontrolvalue('AO2_V',TipBias_vec.tolist())
    time.sleep(0.5)
        
    for k in range(hh_sp_number):
        
        print(str(k*100/hh_sp_number)+'%')
        #Performing scan  
        VI.setcontrolvalue('do_scan', 1)# do scan
        #keep checking if scan is complete
        scanning = True
        while (scanning==True):
            scanning = VI.getcontrolvalue('do_scan')
    
    print("Spiral finished")
    
def Set_Setpoint(Setpoint):

    VI.setcontrolvalue('connected_to_cypher', True);#connect to cypher
    VI.setcontrolvalue('set_contact_setpoint_V', Setpoint);#set setpoint
    VI.setcontrolvalue('set_contact_setpoint', True);#connect to cypher
    
    changing_setpoint = False
    while (changing_setpoint==False):
        changing_setpoint = VI.getcontrolvalue('set_contact_setpoint')
    
    print('Setpoint changed to '+str(Setpoint)+' V')
    
def Square_spiral(center_x,center_y,hh_writting_bias,alter_bias,hh_sp_number,hh_Vx,hh_Vy,hh_cycles,
                   hh_spiral_time,hh_direction,hh_return):
    
    spiral_outer_radius_x = hh_Vx
    spiral_inner_radius_x = 0
    spiral_outer_radius_y = hh_Vy
    spiral_inner_radius_y = 0
    IO_rate = 1E6
    N_cycles = hh_cycles
    duration = hh_spiral_time
    dose_distribution = 1
    direction_opt = hh_direction  # 0 = clockwise spiral out, 1 = counter spiral out, 2 = clockwise spiral in, 3 = counter spiral in
    return_opt = hh_return  # 0 = pigtail return to center, 1 = spiral back to center
    cut_off_frequency = 2000  # [Hz] smooth corners of non-circular spirals

    N_points = int(duration * IO_rate)
    t = np.linspace(0, 1, N_points)
    w = np.linspace(-IO_rate / 2, IO_rate / 2 - IO_rate / N_points, N_points)

    switch_return_opt = return_opt
    switch_direction_opt = direction_opt

    if switch_return_opt == 0:  # pigtail return to center
        amp_vec_x = (spiral_outer_radius_x - spiral_inner_radius_x) * t ** dose_distribution + spiral_inner_radius_x
        amp_vec_y = (spiral_outer_radius_y - spiral_inner_radius_y) * t ** dose_distribution + spiral_inner_radius_y
        x1 = amp_vec_x * np.arcsin(np.sin(2 * np.pi * N_cycles * t))
        y1 = amp_vec_y * np.arcsin(np.cos(2 * np.pi * N_cycles * t))
        a = 1 / N_cycles / 32  # 1/32 of the spiral rotation is spent returning smoothly (erf-like) to the center
        transition_x = (1 - (erf((t - 1 + a * 2) / a) + 1) / 2) * (1 - spiral_inner_radius_x / spiral_outer_radius_x) + spiral_inner_radius_x / spiral_outer_radius_x
        transition_y = (1 - (erf((t - 1 + a * 2) / a) + 1) / 2) * (1 - spiral_inner_radius_y / spiral_outer_radius_y) + spiral_inner_radius_y / spiral_outer_radius_y
        x_vec = x1 * transition_x
        y_vec = y1 * transition_y
    elif switch_return_opt == 1:  # spiral back to center
        tf = t[0::2]
        tr = t[-2::-2]
        amp_vecf_x = (spiral_outer_radius_x - spiral_inner_radius_x) * tf ** dose_distribution + spiral_inner_radius_x
        amp_vecr_x = (spiral_outer_radius_x - spiral_inner_radius_x) * tr ** dose_distribution + spiral_inner_radius_x
        amp_vecf_y = (spiral_outer_radius_y - spiral_inner_radius_y) * tf ** dose_distribution + spiral_inner_radius_y
        amp_vecr_y = (spiral_outer_radius_y - spiral_inner_radius_y) * tr ** dose_distribution + spiral_inner_radius_y
        xf = amp_vecf_x * np.arcsin(np.sin(2 * np.pi * N_cycles * tf))
        xr = -amp_vecr_x * np.arcsin(np.sin(2 * np.pi * N_cycles * tr))
        yf = amp_vecf_y * np.arcsin(np.cos(2 * np.pi * N_cycles * tf))
        yr = amp_vecr_y * np.arcsin(np.cos(2 * np.pi * N_cycles * tr))
        x_vec = np.concatenate((xf, xr))
        y_vec = np.concatenate((yf, yr))

    filter = np.exp(-(w / cut_off_frequency) ** 2)
    filter = np.fft.fftshift(filter)
    x_vec = np.real(np.fft.ifft(np.fft.fft(x_vec) * filter))
    y_vec = np.real(np.fft.ifft(np.fft.fft(y_vec) * filter))

    start_ind = np.argmin(np.abs(x_vec + 1j * y_vec))
    x_vec = np.roll(x_vec, -start_ind + 1) + center_x
    y_vec = np.roll(y_vec, -start_ind + 1) + center_y

    if switch_direction_opt == 1:
        x_vec = -x_vec
    elif switch_direction_opt == 2:
        x_vec = -x_vec
        x_vec = np.flip(x_vec)
        y_vec = np.flip(y_vec)
    elif switch_direction_opt == 3:
        x_vec = np.flip(x_vec)
        y_vec = np.flip(y_vec)

    sector_bias_mat = np.array([[0, 90, hh_writting_bias],
                            [90, 180, -hh_writting_bias],
                            [180, 270, hh_writting_bias],
                            [270, 359.9, -hh_writting_bias]])

    bias_vec = np.zeros_like(x_vec)
    a_vec = np.arctan2(y_vec, x_vec) * 180 / np.pi + 180  # angle of scan path

    for k1 in range(sector_bias_mat.shape[0]):
        ind = np.where((a_vec > sector_bias_mat[k1, 0]) & (a_vec < sector_bias_mat[k1, 1]))
        bias_vec[ind] = sector_bias_mat[k1, 2]
        
    if alter_bias==0:
        bias_vec[:]=hh_writting_bias

    #%% plot trajectory
    plt.figure(1)
    plt.clf()
    plt.plot(x_vec, color='blue')
    plt.plot(y_vec, color='red')
    plt.plot(bias_vec, color='green')
    plt.title('x,y,bias vs. time')

    # plot position over time
    cmap = plt.get_cmap('rainbow')
    N_color_segments = 1000
    colors = cmap(np.linspace(0, 1, N_color_segments))
    plt.figure(2)
    plt.clf()
    line_ind = np.linspace(0, len(x_vec)-1, N_color_segments, dtype = int)
    for i in range(N_color_segments-1):
        plt.plot(x_vec[line_ind[i]:line_ind[i+1]],y_vec[line_ind[i]:line_ind[i+1]],color=colors[i])
    plt.axis('square')
    plt.title('position over time')
    plt.show()

    # plot bias vs position
    if alter_bias!=0:
        colors = cmap(np.linspace(0, 1, N_color_segments))
        plt.figure(2)
        plt.clf()
        line_ind = np.linspace(0, len(x_vec)-1, N_color_segments, dtype = int)
        norm_bias1 = (bias_vec-bias_vec.min())/(bias_vec-bias_vec.min()).max()*(N_color_segments-1)
        norm_bias = norm_bias1.astype(int)
        for i in range(N_color_segments-1):
            plt.plot(x_vec[line_ind[i]:line_ind[i+1]],y_vec[line_ind[i]:line_ind[i+1]],color=colors[norm_bias[line_ind[i]]])
        plt.axis('square')
        plt.title('angle dependent bias')
        plt.show()
    
    #Load the vectors
    #VI.setcontrolvalue('UI_AO0_V',x_vec.tolist())
    #VI.setcontrolvalue('UI_AO1_V',y_vec.tolist())
    #VI.setcontrolvalue('UI_AO2_V',V_vec.tolist())

    #update the scan parameters and wait until finished
    VI.setcontrolvalue('do_scan_update', 1)
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_scan_update')

    #We move tip to the first point to scan
    move_tip_to_pos(center_x,+center_y,1)
    
    # we engage
    Set_Setpoint(Setpoint=1)

    #Perform the scan 
    print('Writting square spiral...')
    for k in range(hh_sp_number):
        #do scan and wait until finished   
        VI.setcontrolvalue('do_scan', 1)
        scanning = True
        while (scanning==True):
            scanning = VI.getcontrolvalue('do_scan')

    # we withdraw
    Set_Setpoint(Setpoint=-10)
    #We move tip back to 0,0
    move_tip_to_pos(0,0,1)
    

## First, we set up a scan to perform the blank slate (bs)

In [ ]:
#%% create user-defined "combing scan" with synchronized tip voltage control
#this is a line-by-line raster scan where bias is applied to tip only when scanning in one direction
VI.setcontrolvalue('scan_type',0) #set scan type to user-defined scan

IO_rate = float(1E6) # IO rate of FPGA [samples/s] - not this could be 10 or 100 times slower
duration = 1 # time to scan one line trace+retrace [Hz]
N_rows = int(256) # number of rows
N_points = int(duration*IO_rate) #number of data points defining output waveforms
scan_size = 7 # extent of scan size [V]
tip_bias = 9 # tip bias [V]

factorY=0.73

t_vec = np.linspace(0., 1., num = N_points,dtype=float) #generate time vector

scan_line_spacing = scan_size/N_rows #distance between scan lines in slow scan direction [V]

x_vec = -scan_size*np.arcsin(np.cos(2.*np.pi*t_vec))/np.pi # x motion during a single line scan
y_vec = scan_line_spacing/4*(np.sign(-t_vec+.5)-1) # y motion during a single line scan - the retrace is half-way between the traces
V_vec = tip_bias/2*(np.sign(-t_vec+.5)+1) # tip bias during a single scan. This is on during the trace and off during the retrace

y_row_pos_vec = factorY*scan_size*np.linspace(-.5, .5, num = N_rows ,dtype=float) # list of row y-positions

# Plot vectors
fig = plt.figure(1)
fig.clf()

# Create the subplots on the figure
ax1 = fig.add_subplot(3, 1, 1)
ax1.plot(t_vec, x_vec)
ax1.set_ylabel('x[V] vs. t[s]')

ax2 = fig.add_subplot(3, 1, 2)
ax2.plot(t_vec, y_vec)
ax2.set_ylabel('y[mV] vs. t[s]')

ax3 = fig.add_subplot(3, 1, 3)
ax3.plot(t_vec, V_vec)
ax3.set_ylabel('V[V] vs. t[s]')

plt.show() # Display the figure

#%%

VI.setcontrolvalue('UI_AO0_V',x_vec.tolist())
VI.setcontrolvalue('UI_AO1_V',y_vec.tolist())
VI.setcontrolvalue('UI_AO2_V',V_vec.tolist())

#First we set up the scan offsets to 0,0:
VI.setcontrolvalue('scan_x_offset_V', 0)
VI.setcontrolvalue('scan_y_offset_V', 0)

#update the scan parameters and wait until finished
VI.setcontrolvalue('do_scan_update', 1)
updating = True
while (updating==True):
    updating = VI.getcontrolvalue('do_scan_update')
    
#We move tip to the center
move_tip_to_pos(0,0,1)

#We move tip to the first point to scan
move_tip_to_pos(x_vec[0],y_row_pos_vec[0],1)
  
# we engage
Set_Setpoint(Setpoint=1)

#cycle through scan offset in the y-direction. Note, we do not need to recreate the scan waveforms for every line 
print('Writting the blank slate...')
cont=-1
for y_row_pos in y_row_pos_vec: 
    
    cont=cont+1
    print(str(cont*100/N_rows)+'%')
    
    VI.setcontrolvalue('scan_y_offset_V', y_row_pos)
   
    #do scan and wait until finished   
    VI.setcontrolvalue('do_scan', 1)
    scanning = True
    while (scanning==True):
        scanning = VI.getcontrolvalue('do_scan')

    #retrieve results - only needed if you want to do imaging during combing    
    #AI_list = VI2.getcontrolvalue('AI_vs_time')    
    # need to parse out the 4 different channels
    # need to average data per pixel
    # then build an image line by line
    # plot results

# we withdraw
Set_Setpoint(Setpoint=-10)

#We move tip back to 0,0
move_tip_to_pos(0,0,1)

## Quick move of probe back to center (in case script is stopped half way)

In [ ]:
move_tip_to_pos(0,0,1)

## Second, perform an array of center convergent/divergent structures

In [ ]:
#Array parameters:
x_hh=1
y_hh=1

#Scanning parameters:
hh_writting_bias=6
hh_Vx=(scan_size/(x_hh+1))/2
hh_Vy=(scan_size/(y_hh+1))/2
hh_cycles=16
hh_spiral_time=1
hh_direction=0 #0clockwise, 1 anticlockwise
hh_return=0 #0 direct return, 1 spiral bck in
hh_sp_number=3

#Calculating parameters of the hedgehog array:
hh_x_pos=np.zeros(x_hh)
hh_y_pos=np.zeros(y_hh)
for i in range(x_hh):
    hh_x_pos[i]=-(scan_size/2)+(i+1)*(scan_size/(x_hh+1))
for i in range(y_hh):
    hh_y_pos[i]=-(scan_size/2)+(i+1)*(scan_size/(y_hh+1))

#we initially move the tip to the 0,0 position
move_tip_to_pos(fin_x=0,fin_y=0,trans_t=1)

#Loop for the array writting
for i in range(x_hh):
    for j in range(y_hh):

        #Moving tip to next position
        move_tip_to_pos(fin_x=hh_x_pos[i],fin_y=hh_y_pos[j],trans_t=1)
        
        # we engage
        Set_Setpoint(Setpoint=1)
        
        #centering the scan offset for spiral scan
        do_write_spiral_scan(hh_x_pos[i],hh_y_pos[j],hh_writting_bias, 
                             hh_sp_number,hh_Vx,hh_Vy,hh_cycles,hh_spiral_time,hh_direction,hh_return)
        
        # we withdraw
        Set_Setpoint(Setpoint=-10)
        
print("Array of Hedgehogs finished")
            
#We move tip back to 0,0
move_tip_to_pos(0,0,1)
time.sleep(1)


## Writting of a single square center divergent/convergent domain

In [ ]:
Square_spiral(center_x=0,center_y=0,hh_writting_bias=-5,alter_bias=1,hh_sp_number=2,hh_Vx=1,hh_Vy=1,hh_cycles=5,
              hh_spiral_time=1,hh_direction=0,hh_return=0)

## Writting of a single  center divergent/convergent domain

In [ ]:
do_write_spiral_scan(0,0,8,60,2.5,factorY*2.5,36,2.5,1,0)

## Automatic PFM reading of the written structures

In [ ]:
#Set parameters for BE
center_freq=335
band_width=100
BE_amp=1
Pha_var=1
repeats=4
pulse_duration=4
auto_smooth=1

#Set parameters for BEPyAE scan
line_num=64
x_start=-1
y_start=-1
x_stop=1
y_stop=-1
BE_filename='test'


newexp = Acquisition(exe_path = exe_path)  # exe_path is the directory of BEPyAE; 
newexp.init_BEPyAE(offline_development = True) # set offline_development=True if doing offline development
                                               # executing this will also initlize AR18
    
newexp.tip_control(tip_parms_dict = {"set_point_V_00": 1, "next_x_pos_00": 0, "next_y_pos_01": 0},
                   do_move_tip = True, 
                   do_set_setpoint = True) # Executing this code will set setpoint to 1 V, 
                                           # and move tip to location [0.5, 0.5]
# set BE parameters
newexp.define_be_parms(be_parms_dict = {"center_frequency_Hz_00": center_freq, "band_width_Hz_01": band_width,
                                       "amplitude_V_02": BE_amp, "phase_variation_03": Pha_var,
                                       "repeats_04": repeats, "req_pulse_duration_s_05": pulse_duration,
                                       "auto_smooth_ring_06": auto_smooth}, 
                      do_create_be_waveform = True)

# Do a 64*64 raster scan
dset_pfm, dset_chns, dset_cs = newexp.raster_scan(raster_parms_dict = {"scan_pixel": line_num, "scan_x_start": x_start,
                                                                       "scan_y_start": y_start,"scan_x_stop": x_stop,
                                                                       "scan_y_stop": y_stop},file_name = BE_filename)

# plot BEPFM quick fit data
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize = (20, 5), dpi = 100)
ax1.imshow(dset_pfm[:,:,0])
ax2.imshow(dset_pfm[:,:,1])
ax3.imshow(dset_pfm[:,:,2])
ax4.imshow(dset_pfm[:,:,3])
plt.show()

## Writting of a single flux-closure domain (flower)

In [ ]:
#%% create user-defined "circle with oscillating radius scan" 

VI.setcontrolvalue('scan_type',0) #set scan type to user-defined scan

IO_rate = float(1E6) # IO rate of FPGA [samples/s] - not this could be 10 or 100 times slower
duration = 1.5 # time to scan [Hz]
N_points = int(duration*IO_rate) #number of data points defining output waveforms
cycle_num=90
# Define the circle's center
center = (0, 0)  # (x, y) coordinates of the center
# Define the Radius
Radius = 0.5
Rad_osc = 40 #define number of radius osiclations
Rad_var = 1 #define radius oscillation variation with respect to full radius
bias_tip=8

t_vec = np.linspace(0., 2*np.pi, num = N_points,dtype=float) #generate time vector

x_vec = center[0] + Radius * (1+(np.cos(Rad_osc*t_vec)/Rad_var))* np.cos(t_vec)  # x motion during a  scan
y_vec = center[1] + Radius * (1+(np.cos(Rad_osc*t_vec)/Rad_var))* np.sin(t_vec) # y motion during a scan
bias_vec = np.linspace(bias_tip, bias_tip, num = N_points,dtype=float) #generate bias vector

y_vec=factorY*y_vec

# Plot vectors
fig = plt.figure(1, figsize=(4,12))
fig.clf()

# Create the subplots on the figure
ax1 = fig.add_subplot(3, 1, 1)
ax1.plot(t_vec, x_vec)
ax1.set_ylabel('x[V] vs. t[s]')

ax2 = fig.add_subplot(3, 1, 2)
ax2.plot(t_vec, y_vec)
ax2.set_ylabel('y[mV] vs. t[s]')

ax3 = fig.add_subplot(3, 1, 3)
ax3.plot(x_vec, y_vec)
ax3.set_ylabel('x[V] vs. y[V]')

plt.show() # Display the figure

#%%

VI.setcontrolvalue('UI_AO0_V',x_vec.tolist())
VI.setcontrolvalue('UI_AO1_V',y_vec.tolist())
VI.setcontrolvalue('UI_AO2_V',bias_vec.tolist())

#First we set up the scan offsets to 0,0:
VI.setcontrolvalue('scan_x_offset_V', 0)
VI.setcontrolvalue('scan_y_offset_V', 0)

#update the scan parameters and wait until finished
VI.setcontrolvalue('do_scan_update', 1)
updating = True
while (updating==True):
    updating = VI.getcontrolvalue('do_scan_update')
    
#We move tip to the center
move_tip_to_pos(0,0,1)

#We move tip to the first point to scan
move_tip_to_pos(x_vec[0],y_vec[0],1)
  
# we engage
Set_Setpoint(Setpoint=1)

#cycle through scan offset in the y-direction. Note, we do not need to recreate the scan waveforms for every line 
print('Writting the circle of oscillating radius...')
for i in range(cycle_num): 
    print(str(i*100/cycle_num)+'%')
    #do scan and wait until finished   
    VI.setcontrolvalue('do_scan', 1)
    scanning = True
    while (scanning==True):
        scanning = VI.getcontrolvalue('do_scan')


# we withdraw
Set_Setpoint(Setpoint=-10)

#We move tip back to 0,0
move_tip_to_pos(0,0,1)